In [1]:
%%bash
pip install kafka-python

In [1]:
from kafka import KafkaConsumer
import json
import csv
import re

In [2]:
consumer = KafkaConsumer(
    bootstrap_servers='localhost:9092',
    auto_offset_reset='earliest',
    value_deserializer=lambda m: json.loads(m.decode('ascii'))
)


In [3]:
consumer.subscribe('jaeger-spans')
for message in consumer:
    # message value and key are raw bytes -- decode if necessary!
    # e.g., for unicode: `message.value.decode('utf-8')`
    # print ("%s:%d:%d: key=%s value=%s" % (message.topic, message.partition, message.offset, message.key, message.value))
    
    
    if message.value['process']['serviceName'] == 'frontend':
        
        # open the file in the append mode
        f = open('/home/vmadmin/log_data.csv', 'a')
        
        # create the csv writer
        writer = csv.writer(f)
        
        http_method = re.findall(r"{'key': 'http\.method', 'vStr': '(.+?)'}",str(message.value))
        http_target = re.findall(r"{'key': 'http\.target', 'vStr': '(.+?)'}",str(message.value))
        http_status_code = re.findall(r"{'key': 'http\.status_code', 'vType': '.*?', 'vInt64': '(\d+)'}",str(message.value))
        http_url = re.findall(r"{'key': 'http\.url', 'vStr': '(.+?)'}",str(message.value))
        
        if len(http_method) > 0:
            http_method = http_method[0]
        if len(http_target) > 0:
            http_target = http_target[0]
        if len(http_status_code) > 0:
            http_status_code = http_status_code[0]
        if len(http_url) > 0:
            http_url = http_url[0]
        
        data = [message.value['traceId'], message.value['spanId'], message.value['operationName'], 
                message.value['duration'], http_method, http_target, http_status_code, http_url]
        
        # write a row to the csv file
        writer.writerow(data)

        # close the file
        f.close()

print("completed")

KeyboardInterrupt: 